<a href="https://colab.research.google.com/github/NadeemSB/IBM_FrontendExperience/blob/main/Passport_OCR.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from imutils import paths
import numpy as np
import argparse
import imutils
import cv2
!sudo apt install tesseract-ocr
!pip install pytesseract

Reading package lists... Done
Building dependency tree       
Reading state information... Done
tesseract-ocr is already the newest version (4.00~git2288-10f4998a-2).
The following package was automatically installed and is no longer required:
  libnvidia-common-460
Use 'sudo apt autoremove' to remove it.
0 upgraded, 0 newly installed, 0 to remove and 49 not upgraded.
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
rectKernel = cv2.getStructuringElement(cv2.MORPH_RECT, (13, 5))
sqKernel = cv2.getStructuringElement(cv2.MORPH_RECT, (21, 21))
image = cv2.imread("/content/passport.jpg")
image = imutils.resize(image, height=600)
gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
gray = cv2.GaussianBlur(gray, (3, 3), 0)
blackhat = cv2.morphologyEx(gray, cv2.MORPH_BLACKHAT, rectKernel)

In [ ]:
gradX = cv2.Sobel(blackhat, ddepth=cv2.CV_32F, dx=1, dy=0, ksize=-1)
gradX = np.absolute(gradX)
(minVal, maxVal) = (np.min(gradX), np.max(gradX))
gradX = (255 * ((gradX - minVal) / (maxVal - minVal))).astype("uint8")
gradX = cv2.morphologyEx(gradX, cv2.MORPH_CLOSE, rectKernel)
thresh = cv2.threshold(gradX, 0, 255, cv2.THRESH_BINARY | cv2.THRESH_OTSU)[1]
thresh = cv2.morphologyEx(thresh, cv2.MORPH_CLOSE, sqKernel)
thresh = cv2.dilate(thresh, sqKernel ,iterations = 2)
thresh = cv2.erode(thresh, None, iterations=2)
cv2.imwrite("/content/result.jpg",thresh)
p = int(image.shape[1] * 0.05)
thresh[:, 0:p] = 0
thresh[:, image.shape[1] - p:] = 0


In [ ]:
cnts = cv2.findContours(thresh.copy(), cv2.RETR_EXTERNAL,cv2.CHAIN_APPROX_SIMPLE)
cnts = imutils.grab_contours(cnts)
cnts = sorted(cnts, key=cv2.contourArea, reverse=True)
for c in cnts:
		# compute the bounding box of the contour and use the contour to
		# compute the aspect ratio and coverage ratio of the bounding box
		# width to the width of the image
	(x, y, w, h) = cv2.boundingRect(c)
	ar = w / float(h)
	crWidth = w / float(gray.shape[1])
		# check to see if the aspect ratio and coverage width are within
		# acceptable criteria
	if ar > 5 and crWidth > 0.75:
			# pad the bounding box since we applied erosions
		pX = int((x + w) * 0.03)
		pY = int((y + h) * 0.03)
		(x, y) = (x - pX, y - pY)
		(w, h) = (w + (pX * 2), h + (pY * 2))
			# extract the ROI from the image and draw a bounding box
			# surrounding the MRZ		
		roi = image[y:y + h, x:x + w].copy()
		cv2.rectangle(image, (x, y), (x + w, y + h), (0, 255, 0), 2)
	# show the output images
cv2.imwrite("/content/Image.jpg", image)
cv2.imwrite("/content/ROI.jpg", roi)

True

# OCR Block

In [ ]:
import pytesseract
!pip install Pillow==9.0.0
import shutil
import os
import random
try:
    from PIL import Image
except ImportError:
    import Image

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
text = pytesseract.image_to_string(Image.open('ROI.jpg'))
print(text)

P<GBROTHER<<ADAM<NORMAN<<<<<<<<<<<<<<<K<<<dd
0740254727GBS6001010M2002020<<<<<<d<Kdd<<<04



# Web Application

In [ ]:
!pip install flask-ngrok
from flask import Flask, render_template
from flask_ngrok import run_with_ngrok

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
from flask import Flask, render_template, request
app = Flask(__name__)
run_with_ngrok(app)
@app.route('/', methods=['GET'])
def index():
    return render_template('/content/drive/MyDrive/Passport_POC/index.html')
@app.route('/post', methods=['POST'])
def post():
    return text
app.run()

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)


 * Running on http://8312-35-231-71-75.ngrok.io
 * Traffic stats available on http://127.0.0.1:4040
